# 2.1.2 Zeichenerklärung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [2]:
from utils.parser import Parser
from utils.openai import OpenAI

instructions = "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
ava = OpenAI(instructions)
parser = Parser()

In [2]:
# Zeichenerklärung (FULL_SIZE, RAW)
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-raw.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg1 = ava.request([
    {
        "type": "text",
        "text": 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg1)

[4625, 8208]
1. **Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung:**

Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter der Überschrift "Planungsrechtliche Festsetzungen".

2. **Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone":**

Die Tabellenerklärung zum Eintrag "Nutzungsschablone" ist direkt unter dem Eintrag zu finden. Sie enthält die Symbole und deren Bedeutungen.

3. **Gebe die Tabellenerklärung in ASCII-Tabellenform aus:**

Hier ist die Tabellenerklärung in ASCII-Tabellenform:

```
+-------------------+------------------------------------------+
| Symbol            | Bedeutung                                |
+-------------------+------------------------------------------+
| G                 | Gewerbegebiete (§ 8 BauNVO)              |
| GE                | Gewerbegebiete eingeschränkt (§ 8 BauNVO)|
+-------------------+------------------------------------------+
```

Diese Tabelle zeigt die Symbole und 

In [3]:
# Zeichenerklärung (SCALED)
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg2 = ava.request([
    {
        "type": "text",
        "text": 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg2)

image_size: [768, 1362]
1. **Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung:**

Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter der Kategorie "Art der Nutzung".

2. **Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone":**

Die Tabellenerklärung zum Eintrag "Nutzungsschablone" ist direkt unter dem Eintrag zu finden und lautet:
- Nutzungsschablone (§ 9 (1) 1 BauGB)

3. **Gebe die Tabellenerklärung in ASCII-Tabellenform aus:**

```plaintext
+-------------------+-----------------------------+
| Nutzungsschablone | (§ 9 (1) 1 BauGB)           |
+-------------------+-----------------------------+
```

Dies ist die ASCII-Tabellenform der Tabellenerklärung für den Eintrag "Nutzungsschablone".


In [4]:
# Zeichenerklärung (SCALED, CROPPED)
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg3 = ava.request([
    {
        "type": "text",
        "text": 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg3)

image_size: [768, 1949]
1. Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter "Art der Nutzung" und "Gebäudehöhe".

2. Die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone" ist in der Tabelle direkt darunter zu finden.

3. Die Tabellenerklärung in ASCII-Tabellenform:

```
+----------------+----------------+
| Art der Nutzung| Gebäudehöhe    |
+----------------+----------------+
| Grundflächenzahl| Geschossflächen-|
|                | zahl           |
+----------------+----------------+
| Bauweise       | Nutzungsschablone|
|                | (§ 9 (1) 1 BauGB)|
+----------------+----------------+
```

Diese Tabelle zeigt die verschiedenen Kategorien und deren zugehörige Werte, einschließlich der Nutzungsschablone gemäß § 9 (1) 1 BauGB.


In [3]:
# Zeichenerklärung (BOUNDING BOX, CROPPED)
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-bb-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg4 = ava.request([
    {
        "type": "text",
        "text": 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg4)

image_size: [763, 2000]
Hier ist die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform:

```
+---------------------+------------------+
| Art der Nutzung     | Gebäudehöhe      |
+---------------------+------------------+
| Grundflächenzahl    | Geschossflächen- |
|                     | zahl             |
+---------------------+------------------+
| Bauweise            |                  |
+---------------------+------------------+
```


In [9]:
# Zeichenerklärung (SCALED, CROPPED)
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

ava.addContext([
    {
        "type": "text",
        "text": msg4
    }
])

msg5 = ava.request([
    {
        "type": "text",
        "text": 'Fasse alle relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Parameter>: {"Definition": "", "Beispiel"=""}}'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg5)
ava.clearContext()

image_size: [768, 1949]
Hier sind die relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone im JSON-Format:

```json
{
  "Art der Nutzung": {
    "Definition": "Gewerbegebiete (§ 8 BauNVO) oder Gewerbegebiete eingeschränkt (§ 8 BauNVO)",
    "Beispiel": "GE, GEE"
  },
  "Gebäudehöhe": {
    "Definition": "Maximal zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe",
    "Beispiel": "GH=max.12,00"
  },
  "Grundflächenzahl": {
    "Definition": "Grundflächenzahl (GRZ) gemäß § 16 BauNVO",
    "Beispiel": "0,6"
  },
  "Geschossflächenzahl": {
    "Definition": "Geschossflächenzahl (GFZ) gemäß § 16 BauNVO",
    "Beispiel": "1,2"
  },
  "Bauweise": {
    "Definition": "Abweichende Bauweise gemäß § 22 BauNVO",
    "Beispiel": "a"
  }
}
```
